In [3]:
!pip install pyTelegramBotAPI

import telebot
from telebot import types
import pandas as pd
import numpy as np

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

from keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk import word_tokenize, pos_tag

from string import punctuation
import re


import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en

import spacy

!pip install stop-words
from stop_words import get_stop_words

from textblob import TextBlob

from bs4 import BeautifulSoup
import requests
import json
from urllib.parse import urlparse


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 12.0 MB 5.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=10418d6f04898fd9a8ae597163e6a62495d19bad6aa54d89c624580cef251c7e
  Stored in dire

In [4]:
bot = telebot.TeleBot('5260450850:AAGkp80bHug3CzO0AVj41IEoVNDocOJLfGk');

In [5]:

@bot.message_handler(commands=["start"])
def start(m, res=False):
        markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
        item1=types.KeyboardButton("Начать")
        item2=types.KeyboardButton("Информация")
        item3=types.KeyboardButton("Помощь")
        markup.add(item1)
        markup.add(item2)
        markup.add(item3)
        bot.send_message(m.chat.id, 'Нажми: \nНачать(чтобы воспользоваться программой)\nИнформация(чтобы узанть о нас)\nПомощь(чтобы узнать о боте)',  reply_markup=markup)

In [7]:

@bot.message_handler(func=lambda message: message.text=='Начать')
def rabota(message):
  bot.send_message(message.from_user.id, 'Введите ссылку на новость')
  bot.register_next_step_handler(message, test);
def test(message):
    bot.send_message(message.chat.id, text=message.text)
    link=message.text
    req = requests.get(link).text
    soup = BeautifulSoup(req, "lxml")
    head = soup.find('head')
    zagolovok = head.find('title').text
    textp = soup.find_all('div', attrs={'class': "article__text"})
    if (len(textp)) == 0:
        textp = soup.find_all('span', attrs={'class': 'intro'})
    if (len(textp)) == 0:
        text = soup.find_all('span', attrs={'class': 'intro'})
    if (len(textp)) == 0:
        textp = soup.find_all('p')
    
    domain = urlparse(link).netloc
    if (int(domain.find("www.")) != -1):
      domain = domain[4:]
   
    url = link.split("//")[-1].split("/")[0].split('?')[0]
    show = "https://input.payapi.io/v1/api/fraud/domain/age/" + domain

    r = requests.get(show, verify = False)
    soupp = BeautifulSoup(r.text, "lxml")

    if requests.get(link).status_code ==200:

        age = soupp.find('body').text
        str = int(age.find("Reg"))
        end = int(age.find("}"))
        # print(age[str:(end - 1)])
        age=age[str:(end - 1)]
        data = r.text
        jsonToPython = json.loads(data)
    else:
      age=np.nan
    for j in range(len(textp)):
      textp[j]=textp[j].text
    list = [zagolovok, textp, age, domain,requests.get(link).status_code]
    bot.send_message(message.chat.id,text=list)
def proga(m):
  bot.send_message(m.from_user.id, 'ссылка обрабатывается')
@bot.message_handler(func=lambda message: message.text=='Информация')
def info(message):
  bot.send_message(message.from_user.id, "Это проект учащихся IT-класса лицея №1511 при МИФИ, созданный для защиты от фейковых новостей в сети.")
@bot.message_handler(func=lambda message: message.text=='Помощь')
def help(message):
  bot.send_message(message.from_user.id, "Если у вас есть вопросы по боту или работе программы, то задать их можно здесь @miCherex")

bot.polling() 